In [ ]:
def save_notification_to_db(
            self,
            table: str = "public.customer_notifications",
            user_id: str = "",
            insight_name: str = "",
            notification_name: str = "",
            date: str = "",
            sent_to_user: bool = False,
            job_execution_uid: int = None,
            verbose: int = 0
    ):
    import psycopg2

    # Database connection parameters
    db_name = "postgres"
    db_user = "postgres"
    db_password = XXX
    db_host = "datawarehouse-dev.cwjw1iv5xvhh.us-west-2.rds.amazonaws.com"
    db_port = "5432"

    # Connect to the database
    conn = psycopg2.connect(
        dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port
    )

    # Create a cursor object
    cur = conn.cursor()

    sql_sent_to_user = "TRUE" if sent_to_user else "FALSE"

    # SQL query to insert a new row and return the colId
    insert_query = f"""
    INSERT INTO customer_notifications (id, customer_uuid, flg_message_sent_customer, insight_catalogue_id, insight_execution_log_id, notification_dictionary_id, time_zone_id, last_updated, customer_age, gender_id, country_id, country_state_id, dow, hod, date_record_utc)
    SELECT (SELECT COUNT(id) + 1 FROM customer_notifications) AS id,
    '{user_id}' AS customer_uuid,
    {sql_sent_to_user} AS flg_message_sent_customer,
    (SELECT id FROM insight_catalogue WHERE insight_name='{insight_name}') AS insight_catalogue_id,
    CAST('{job_execution_uid}' AS BIGINT) AS insight_execution_log_id,
    (SELECT id FROM notification_dictionary WHERE notification_name='{notification_name}') AS notification_dictionary_id,
    cdata.time_zone_id AS time_zone_id,
    '{date}' AS last_updated,
    (SELECT (EXTRACT(YEAR FROM AGE(CURRENT_DATE, (CONCAT(cdata.yob,'-07-01'))::DATE))) FROM customer_data AS cdata WHERE cdata.yob<>'0' AND cdata.customer_uuid='{user_id}' LIMIT 1) AS customer_age,
    cdata.gender_id AS gender_id,
    cdata.country_id AS country_id,
    cdata.country_state_id AS country_state_id,
    EXTRACT(DOW FROM CURRENT_TIMESTAMP AT TIME ZONE 'UTC') % 7 + 1 AS dow,
    EXTRACT(HOUR FROM CURRENT_TIMESTAMP AT TIME ZONE 'UTC') AS hod,
    CURRENT_TIMESTAMP AT TIME ZONE 'UTC' AS date_record_utc

    FROM
        customer_data cdata

    WHERE cdata.customer_uuid='{user_id}' LIMIT 1

    RETURNING *;
    """


    # Execute the query
    cur.execute(insert_query)

    # Fetch the returned colId
    colId_value = cur.fetchone()

    # Commit the changes
    conn.commit()

    # Print the colId value
    dataframe_to_save=colId_value

    # Close the cursor and connection
    cur.close()
    conn.close()
    return dataframe_to_save

In [ ]:
linea=save_notification_to_db(1,
                    user_id="03459abe-1189-4d6d-ba69-14265d3d575a",
                    insight_name='Heart Health',
                    notification_name="Non executed",
                    sent_to_user=True,
                    date='2023-09-14',
                    job_execution_uid="5",
                    verbose=True
                )
print(linea)


(31, '03459abe-1189-4d6d-ba69-14265d3d575a', 35, 1, None, None, None, None, None, 4, '8', datetime.datetime(2024, 1, 31, 8, 42, 48, 50619), None, 50, None, None, 4, 5, None, None, datetime.datetime(2023, 9, 14, 0, 0, tzinfo=datetime.timezone.utc), True, None)
